# LOB Forecasting + RL Demo

In [ ]:
import numpy as np, pandas as pd
from src.data_loader import LOBDataLoader
from src.models.baseline import Baselines
from src.backtester import Backtester
# Optional PPO
try:
    from src.rl.train_agent import train_ppo_agent
except Exception:
    train_ppo_agent = None

# Load sample (replace with FI-2010 once downloaded to data/)
loader = LOBDataLoader("data/LOB_sample.csv")
loader.load_data(); loader.compute_features(); loader.label_data(horizon=10)
df = loader.df
features = df[["spread","mid_price","ofi","depth_imb","rolling_vol"]]
labels = df["label"]; mid_prices = df["mid_price"]

# Baseline
bl = Baselines()
res = bl.train_logreg(features, labels, mid_prices)
print(f"LogReg Acc={res.acc:.4f} F1={res.f1:.4f} Sharpe={res.sharpe:.4f}")

# Backtest baseline signals
bt = Backtester()
eq = bt.run(mid_prices.loc[res.preds.index], res.preds)
print(bt.compute_metrics())

# PPO (optional - comment out if sb3 not installed)
# if train_ppo_agent is not None:
#     model = train_ppo_agent(features.values.astype(np.float32), mid_prices.values, preds=res.preds.values, total_timesteps=5000)
#     model.save("ppo_lob_agent.zip")
